# Microsoft Movie Needs Analysis

[Insert image here]

## Overview

### Business Problem

## Data Understanding

In [1]:
import pandas as pd
import numpy as np
import datetime as datetime

In [2]:
# name = pd.read_csv("unzippedData/imdb.name.basics.csv")
# title_akas  = pd.read_csv("unzippedData/imdb.title.akas.csv")
# tmdb_movies  = pd.read_csv("unzippedData/tmdb.movies.csv")
title_basics  = pd.read_csv("unzippedData/imdb.title.basics.csv")
# title_principals  = pd.read_csv("unzippedData/imdb.title.principals.csv")
ratings = pd.read_csv('imdb.title.ratings.csv.gz')

In [3]:
movies_info = title_basics.merge(ratings, how='left', on='tconst')
movies_info.rename({'primary_title' : 'movie', 
                    'start_year' : 'release_year'}, axis =1, inplace= True)
movies_info['release_year'] = movies_info['release_year'].astype(int)
movies_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146144 entries, 0 to 146143
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           146144 non-null  object 
 1   movie            146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   release_year     146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
 6   averagerating    73856 non-null   float64
 7   numvotes         73856 non-null   float64
dtypes: float64(3), int64(1), object(4)
memory usage: 10.0+ MB


In [4]:
movie_budgets = pd.read_csv('unzippedData/tn.movie_budgets.csv')
movie_budgets

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [5]:
movie_budgets['release_year'] = movie_budgets['release_date'].map(lambda x: int(x[-4:]))


In [6]:
# lambda function starts index at 1 to remove $ in value, converts to int 
movie_budgets['worldwide_gross'] = movie_budgets['worldwide_gross'].map(lambda x: int(x[1:].replace(",","")))
movie_budgets['production_budget'] = movie_budgets['production_budget'].map(lambda x: int(x[1:].replace(",","")))
movie_budgets['domestic_gross'] = movie_budgets['domestic_gross'].map(lambda x: int(x[1:].replace(",","")))

In [7]:
movie_budgets["domestic_profit"] = movie_budgets["domestic_gross"] - movie_budgets["production_budget"]
movie_budgets["worldwide_profit"] = movie_budgets["worldwide_gross"] - movie_budgets["production_budget"]

In [8]:
movie_budgets["domestic_roi"] = movie_budgets["domestic_profit"]/movie_budgets['production_budget']
movie_budgets["worldwide_roi"] = movie_budgets["worldwide_profit"]/movie_budgets["production_budget"]

In [9]:
movie_budgets.describe()

,id,production_budget,domestic_gross,worldwide_gross,release_year,domestic_profit,worldwide_profit,domestic_roi,worldwide_roi
count,5782.000000,5.782000e+03,5.782000e+03,5.782000e+03,5782.000000,5.782000e+03,5.782000e+03,5782.000000,5782.000000
mean,50.372363,3.158776e+07,4.187333e+07,9.148746e+07,2003.967139,1.028557e+07,5.989970e+07,1.983540,3.800161
std,28.821076,4.181208e+07,6.824060e+07,1.747200e+08,12.724386,4.992137e+07,1.460889e+08,26.086591,29.530282
min,1.000000,1.100000e+03,0.000000e+00,0.000000e+00,1915.000000,-3.072376e+08,-2.002376e+08,-1.000000,-1.000000
25%,25.000000,5.000000e+06,1.429534e+06,4.125415e+06,2000.000000,-9.132757e+06,-2.189071e+06,-0.794367,-0.507704
50%,50.000000,1.700000e+07,1.722594e+07,2.798445e+07,2007.000000,-3.487755e+05,8.550286e+06,-0.121674,0.708310
75%,75.000000,4.000000e+07,5.234866e+07,9.764584e+07,2012.000000,1.778144e+07,6.096850e+07,1.058148,2.758346
max,100.000000,4.250000e+08,9.366622e+08,2.776345e+09,2020.000000,6.306622e+08,2.351345e+09,1799.000000,1799.000000


In [10]:
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5782 non-null   int64  
 1   release_date       5782 non-null   object 
 2   movie              5782 non-null   object 
 3   production_budget  5782 non-null   int64  
 4   domestic_gross     5782 non-null   int64  
 5   worldwide_gross    5782 non-null   int64  
 6   release_year       5782 non-null   int64  
 7   domestic_profit    5782 non-null   int64  
 8   worldwide_profit   5782 non-null   int64  
 9   domestic_roi       5782 non-null   float64
 10  worldwide_roi      5782 non-null   float64
dtypes: float64(2), int64(7), object(2)
memory usage: 497.0+ KB


In [11]:
movie_budgets_cleaned = movie_budgets[(movie_budgets['worldwide_roi']>=5) 
             & ((movie_budgets['production_budget']>=5000000)
             & (movie_budgets['worldwide_gross']>=2))
              & (movie_budgets['release_year'] != 2020) 
                & (movie_budgets['release_year'] >= 2010)]

In [12]:
movies_df = movie_budgets_cleaned.merge(movies_info, how='left', on =['movie','release_year'])
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196 entries, 0 to 195
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 196 non-null    int64  
 1   release_date       196 non-null    object 
 2   movie              196 non-null    object 
 3   production_budget  196 non-null    int64  
 4   domestic_gross     196 non-null    int64  
 5   worldwide_gross    196 non-null    int64  
 6   release_year       196 non-null    int64  
 7   domestic_profit    196 non-null    int64  
 8   worldwide_profit   196 non-null    int64  
 9   domestic_roi       196 non-null    float64
 10  worldwide_roi      196 non-null    float64
 11  tconst             157 non-null    object 
 12  original_title     157 non-null    object 
 13  runtime_minutes    157 non-null    float64
 14  genres             157 non-null    object 
 15  averagerating      156 non-null    float64
 16  numvotes           156 non

In [13]:
# movies_df.dropna(subset=['averagerating'], inplace=True)
# movies_df = movies_df.fillna(0)
# movies_df['start_year_int'] = movies_df['start_year'].astype(int)
# movies_df['release_year'] = movies_df['release_date'].map(lambda x: int(x[-4:]))
# movies_df[(movies_df['release_year'] == (movies_df['start_year_int'])) 
#      & (movies_df['original_title'] == movies_df['movie'] )]

In [14]:
movies_df.drop(movies_df[movies_df['release_year'] > 2019].index, inplace=True)
movies_df['release_year'].min()

2010

In [19]:
movies_df = movies_df[movies_df['movie']== movies_df['original_title']]

In [20]:
movies_df['movie'].value_counts()

Lights Out                                   2
The Artist                                   2
Paranormal Activity 4                        1
Sultan                                       1
Paranormal Activity: The Ghost Dimension     1
                                            ..
The First Purge                              1
Three Billboards Outside Ebbing, Missouri    1
Lion                                         1
Mama                                         1
Baby Driver                                  1
Name: movie, Length: 153, dtype: int64

In [43]:
movies_df[movies_df['movie']=='As Above, So Below']

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,domestic_profit,worldwide_profit,domestic_roi,worldwide_roi,tconst,original_title,runtime_minutes,genres,averagerating,numvotes
192,84,"Aug 29, 2014","As Above, So Below",5000000,21222315,41824066,2014,16222315,36824066,3.244463,7.364813,tt2870612,"As Above, So Below",93.0,"[Adventure, Horror, Mystery]",6.2,66892.0


In [21]:
movies_info[movies_info['movie']=='Lights Out']

,tconst,movie,original_title,release_year,runtime_minutes,genres,averagerating,numvotes
5536,tt1085448,Lights Out,Lights Out,2010,NaN,Drama,6.6,10.0
11911,tt1646127,Lights Out,Simon Werner a disparu...,2010,87.0,"Drama,Mystery,Thriller",6.4,971.0
43856,tt2611518,Lights Out,Lights Out,2013,NaN,Drama,8.6,22.0
82941,tt4786282,Lights Out,Lights Out,2016,81.0,"Drama,Horror,Mystery",6.3,100650.0
92110,tt5328340,Lights Out,Lights Out,2016,90.0,Documentary,NaN,NaN


In [79]:
movies_df[movies_df['movie']=='Dallas Buyers Club']

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,domestic_profit,worldwide_profit,domestic_roi,worldwide_roi,tconst,original_title,runtime_minutes,genres,averagerating,numvotes
189,73,"Nov 1, 2013",Dallas Buyers Club,5000000,27298285,60611845,2013,22298285,55611845,4.459657,11.122369,tt0790636,Dallas Buyers Club,117.0,"[Biography, Drama]",8.0,402462.0


In [52]:
movies_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,domestic_profit,worldwide_profit,domestic_roi,worldwide_roi,tconst,original_title,runtime_minutes,genres,averagerating,numvotes
1,7,"Apr 27, 2018",Avengers: Infinity War,300000000,678815482,2048134200,2018,378815482,1748134200,1.262718,5.827114,tt4154756,Avengers: Infinity War,149.0,"[Action, Adventure, Sci-Fi]",8.5,670926.0
2,27,"May 4, 2012",The Avengers,225000000,623279547,1517935897,2012,398279547,1292935897,1.770131,5.746382,tt0848228,The Avengers,143.0,"[Action, Adventure, Sci-Fi]",8.1,1183655.0
3,34,"Jun 12, 2015",Jurassic World,215000000,652270625,1648854864,2015,437270625,1433854864,2.033817,6.669092,tt0369610,Jurassic World,124.0,"[Action, Adventure, Sci-Fi]",7.0,539338.0
4,42,"Feb 16, 2018",Black Panther,200000000,700059566,1348258224,2018,500059566,1148258224,2.500298,5.741291,tt1825683,Black Panther,134.0,"[Action, Adventure, Sci-Fi]",7.3,516148.0
5,44,"Jun 15, 2018",Incredibles 2,200000000,608581744,1242520711,2018,408581744,1042520711,2.042909,5.212604,tt3606756,Incredibles 2,118.0,"[Action, Adventure, Animation]",7.7,203510.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,70,"Jan 3, 2014",Paranormal Activity: The Marked Ones,5000000,32462372,90904854,2014,27462372,85904854,5.492474,17.180971,tt2473682,Paranormal Activity: The Marked Ones,84.0,"[Horror, Mystery, Thriller]",5.0,37147.0
189,73,"Nov 1, 2013",Dallas Buyers Club,5000000,27298285,60611845,2013,22298285,55611845,4.459657,11.122369,tt0790636,Dallas Buyers Club,117.0,"[Biography, Drama]",8.0,402462.0
190,74,"Aug 26, 2015",No Escape,5000000,27288872,42844516,2015,22288872,37844516,4.457774,7.568903,tt1781922,No Escape,103.0,"[Action, Thriller]",6.8,70222.0
191,76,"Feb 27, 2015",The Lazarus Effect,5000000,25801570,38359310,2015,20801570,33359310,4.160314,6.671862,tt2918436,The Lazarus Effect,83.0,"[Horror, Mystery, Sci-Fi]",5.2,37675.0


In [68]:
list(movies_df.columns)

['id',
 'release_date',
 'movie',
 'production_budget',
 'domestic_gross',
 'worldwide_gross',
 'release_year',
 'domestic_profit',
 'worldwide_profit',
 'domestic_roi',
 'worldwide_roi',
 'tconst',
 'original_title',
 'runtime_minutes',
 'genres',
 'averagerating',
 'numvotes']

In [72]:
columns = list(movies_df.columns)
columns.remove('genres')
columns

['id',
 'release_date',
 'movie',
 'production_budget',
 'domestic_gross',
 'worldwide_gross',
 'release_year',
 'domestic_profit',
 'worldwide_profit',
 'domestic_roi',
 'worldwide_roi',
 'tconst',
 'original_title',
 'runtime_minutes',
 'averagerating',
 'numvotes']

In [80]:
# movies_df['genres'] =movies_df['genres'].map(lambda x: x.split(","))
# movies_df['genres'].apply(pd.Series).reset_index().melt(id_vars='index').dropna()[['index', 'value']].set_index('index')

movies_df = movies_df.genres.apply(pd.Series) \
    .merge(movies_df, right_index = True, left_index = True) \
    .drop(["genres"], axis = 1) \
    .melt(id_vars = columns , value_name = "genres") \
    .dropna()


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,domestic_profit,worldwide_profit,domestic_roi,worldwide_roi,tconst,original_title,runtime_minutes,averagerating,numvotes,variable,genres
151,73,"Nov 1, 2013",Dallas Buyers Club,5000000,27298285,60611845,2013,22298285,55611845,4.459657,11.122369,tt0790636,Dallas Buyers Club,117.0,8.0,402462.0,0,Biography
306,73,"Nov 1, 2013",Dallas Buyers Club,5000000,27298285,60611845,2013,22298285,55611845,4.459657,11.122369,tt0790636,Dallas Buyers Club,117.0,8.0,402462.0,1,Drama
